In [1]:
import torch
from itertools import combinations
import numpy as np
import torch.nn.functional as F
from typing import List, Tuple, Dict, Any
import matplotlib.pyplot as plt

In [2]:
from fuzz.utils import enumerate_permute, gap_count
from fuzz.src.capacity import *
from fuzz.src.norm import *
from fuzz.src.choquet.choquet import *
from fuzz.src.choquet.utils import *
from fuzz.src.sim import *
from fuzz.src.knn import *
from fuzz.eval import FuzzLOO

### utils.py

In [3]:
x = torch.rand(300, 4)
print("done")
enum = enumerate_permute(x[1].unsqueeze(0))
print(enum)
print("shape", enum.shape)
gap_count(enum[0][0])

done
tensor([[[-1, -1, -1, -1],
         [ 1, -1, -1, -1],
         [ 2, -1, -1, -1],
         [ 0, -1, -1, -1],
         [ 3, -1, -1, -1],
         [ 1,  2, -1, -1],
         [ 1,  0, -1, -1],
         [ 1,  3, -1, -1],
         [ 2,  0, -1, -1],
         [ 2,  3, -1, -1],
         [ 0,  3, -1, -1],
         [ 1,  2,  0, -1],
         [ 1,  2,  3, -1],
         [ 1,  0,  3, -1],
         [ 2,  0,  3, -1],
         [ 1,  2,  0,  3]]])
shape torch.Size([1, 16, 4])


4

### capacity.py

In [4]:
mu = generate_capacity(enum[0])
for i in range(len(mu)):
    print(mu[i].get_capacity())

print(f"Capacity locator for {enum[0][4]}:", locate_capacity(enum[0][4], mu))

0
0.018278539180755615
0.23042798042297363
0.2890784740447998
0.29427945613861084
0.30455321073532104
0.32746708393096924
0.45462852716445923
0.47011882066726685
0.5352305173873901
0.7589643001556396
0.8414394855499268
0.8701643347740173
0.9690427184104919
0.9793176651000977
1
Capacity locator for tensor([ 3, -1, -1, -1]): 0.29427945613861084


### norm.py

In [5]:
x = normalize(x)
print("Normalized x:\n", x)

Normalized x:
 tensor([[0.6561, 0.0000, 0.2513, 1.0000],
        [0.7168, 0.0000, 0.5245, 1.0000],
        [1.0000, 0.8301, 0.5730, 0.0000],
        ...,
        [0.0000, 0.1835, 1.0000, 0.8386],
        [0.9158, 1.0000, 0.0000, 0.5131],
        [0.4989, 1.0000, 0.0000, 0.2628]])


In [6]:
x2 = torch.rand(300,4)
x2 = normalize(x2)
x2

tensor([[0.8225, 0.7629, 0.0000, 1.0000],
        [0.1195, 0.0000, 1.0000, 0.2769],
        [1.0000, 0.7784, 0.0000, 0.1117],
        ...,
        [0.6559, 0.0000, 1.0000, 0.1558],
        [1.0000, 0.8557, 0.2585, 0.0000],
        [0.3959, 0.0000, 0.6103, 1.0000]])

In [7]:
T_norm_result = T_conorm(x, x2, mode='L')
T_norm_result

tensor([[1.0000, 0.7629, 0.2513, 1.0000],
        [0.8363, 0.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 0.5730, 0.1117],
        ...,
        [0.6559, 0.1835, 1.0000, 0.9943],
        [1.0000, 1.0000, 0.2585, 0.5131],
        [0.8948, 1.0000, 0.6103, 1.0000]])

### choquet.py

In [8]:
s_union_result = s_triangle(x, x2, mode='M')
s_union_result

tensor([[0.3439, 0.7629, 0.2513, 0.0000],
        [0.7168, 0.0000, 0.4755, 0.7231],
        [0.0000, 0.2216, 0.5730, 0.1117],
        ...,
        [0.6559, 0.1835, 0.0000, 0.8386],
        [0.0842, 0.1443, 0.2585, 0.5131],
        [0.4989, 1.0000, 0.6103, 0.7372]])

In [9]:
x[0]

tensor([0.6561, 0.0000, 0.2513, 1.0000])

In [10]:
enumerate_permute(x[0].unsqueeze(0))

tensor([[[-1, -1, -1, -1],
         [ 1, -1, -1, -1],
         [ 2, -1, -1, -1],
         [ 0, -1, -1, -1],
         [ 3, -1, -1, -1],
         [ 1,  2, -1, -1],
         [ 1,  0, -1, -1],
         [ 1,  3, -1, -1],
         [ 2,  0, -1, -1],
         [ 2,  3, -1, -1],
         [ 0,  3, -1, -1],
         [ 1,  2,  0, -1],
         [ 1,  2,  3, -1],
         [ 1,  0,  3, -1],
         [ 2,  0,  3, -1],
         [ 1,  2,  0,  3]]])

In [ ]:
choquet = Choquet(X=x[0].unsqueeze(0), mu=mu)

In [ ]:
choquet.choquet

### sim.py

In [ ]:
s_union_result = s_triangle(x, x2, mode='M')
s_union_result[0].unsqueeze(0)

In [ ]:
choquet2 = Choquet(X=s_union_result[0].unsqueeze(0), mu=mu)
choquet2.choquet

In [ ]:
for i in range(5):
    sim = S1(X=x[0].unsqueeze(0), Y=x2[i].unsqueeze(0), mu=mu)
    print("Sim1:", sim.score())
    sim = S2(X=x[0].unsqueeze(0), Y=x2[i].unsqueeze(0), mu=mu)
    print("Sim2:", sim.score())
    sim = S3(X=x[0].unsqueeze(0), Y=x2[i].unsqueeze(0), mu=mu)
    print("Sim3:", sim.score())
    print()

### knn.py

In [ ]:
label = torch.randint(0, 2, (300, ))

In [ ]:
knn = KNNFuzz(input_dimension=x.size(1), mu=mu, k=3, sim=S1)
knn.fit(desc_set=x, label_set=label)
knn.accuracy(desc_set=x, label_set=label)

In [ ]:
import time 
import copy

In [ ]:
X = x 
Y = label

In [ ]:
C = KNNFuzz(input_dimension=4, mu=mu, k=3, sim=S1)

In [ ]:
FuzzLOO(
    C = C,
    DS = (x, label),
    mu = mu,
    time_counter=True
)

### eval.py

In [ ]:
# LOO evaluation
# Define list for each sim level
l1 = []
l2 = []
l3 = []
k = 10

for k in range(1, k+1): 
    tic = time.time()
    mu = generate_capacity(enumerate_permute(x[0].unsqueeze(0))[0])

    print(f"Leave one out avec k = {k}")
    
    # SimLevel1
    res = FuzzLOO(
            C = KNNFuzz(input_dimension=x.size(1),mu=mu, k=k, sim=S1), 
            DS = (x, label),
            mu=mu
        )
    l1.append(res)
    print(f"SimLevel1: {res}")
    # Sim level 2
    res = FuzzLOO(
            C = KNNFuzz(input_dimension=x.size(1),mu=mu, k=k, sim=S2), 
            DS = (x, label),
            mu=mu
        )
    l2.append(res)
    print(f"SimLevel2: {res}")

    # Sim level 3
    res = FuzzLOO(
            C = KNNFuzz(input_dimension=x.size(1),mu=mu, k=k, sim=S3), 
            DS = (x, label),
            mu=mu
        )
    l3.append(res)
    print(f"SimLevel3: {res}\n")

    toc = time.time()
    print(f"Result in {(toc-tic):0.4f} seconds.")

# Plot results
plt.plot(l1, label="Sim level 1", marker='o')
plt.plot(l2, label="Sim level 2", marker='o')
plt.plot(l3, label="Sim level 3", marker='o')

# Config additional params
plt.ylabel("LOO accuracy")
plt.xlabel("k")
plt.title("Leave-One-Out Evaluation Results")
plt.grid()
plt.legend()

In [11]:
nca = NCAFuzzKNN(
    input_dimension=x.size(1),
    mu = mu,
    k = 3,
    sim = S1,
    output_dimension=3
)
transformed_x = nca.transform(x)
print(transformed_x.size())
transformed_x = normalize(transformed_x)

# Regenrate mu for transformed data
mu_transformed = generate_capacity(enumerate_permute(transformed_x[0].unsqueeze(0))[0])

torch.Size([300, 3])


In [12]:
label = torch.randint(0, 2, (300, ))

In [13]:
# loss pretesting
loss = nca.loss(x, label)
print(loss)
print(f"loss.grad_fn: {loss.grad_fn}")


KeyboardInterrupt: 

In [14]:
for name, param in nca.nca.named_parameters():
    print(f"{name} requires_grad = {param.requires_grad}")

A requires_grad = True


In [15]:
losses = nca.fit(
    desc_set=x, 
    label_set=label,
    num_epochs=100,
    learning_rate=0.01,
    batch_size=32,
)


loss: -15.567789077758789
loss: -18.255142211914062
loss: -15.595266342163086
loss: -16.093446731567383
loss: -16.050168991088867
loss: -17.19662857055664
loss: -15.629596710205078
loss: -15.489431381225586
loss: -15.52365493774414
loss: -7.129511833190918
loss: -15.493743896484375
loss: -15.551420211791992
loss: -15.44131851196289
loss: -15.460970878601074
loss: -15.509842872619629
loss: -18.713180541992188
loss: -16.459930419921875
loss: -16.611608505249023
loss: -15.752582550048828
loss: -5.9661736488342285
loss: -15.447214126586914
loss: -16.05492401123047
loss: -17.13758087158203
loss: -16.086931228637695
loss: -15.428831100463867
loss: -16.22800064086914
loss: -15.351184844970703
loss: -15.724462509155273
loss: -16.40548324584961
loss: -8.340005874633789
loss: -15.552774429321289
loss: -16.900108337402344
loss: -15.489665985107422
loss: -15.600521087646484
loss: -15.37868881225586
loss: -15.51361083984375
loss: -15.252984046936035
loss: -16.06177520751953
loss: -15.49724388122558

KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(losses, label="Training Loss", marker='o')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss Over Epochs")
plt.grid()

#### Test compute pij

In [ ]:
pij = nca.compute_pij(transformed_x, mu_transformed)

In [ ]:
test = pij.numpy()
import seaborn as sns
sns.set(style="white")
plt.figure(figsize=(10, 8))
sns.heatmap(test, fmt=".2f", cmap='coolwarm')

#### NCA loss

In [ ]:
optimizer = torch.optim.Adam([nca.nca.A], lr=1e-3)

In [ ]:
for i in range(5):
    print("Epoch", i)
    epoch_loss = 0
    # Shuffle the data for mini-batch training
    indices = torch.randperm(10)

    # Mini-batch training
    for start_idx in range(0, 10, 21):
        # Get mini-batch indices
        batch_indices = indices[start_idx:min(start_idx + 21, 10)]

        # Get batch data
        batch_x = x[batch_indices]
        batch_y = label[batch_indices]

        # Regenerate mu for transformed data
        mu_transformed = generate_capacity(enumerate_permute(batch_x[0].unsqueeze(0))[0])

        # Zero the gradients
        # optimizer.zero_grad()

        # Compute loss
        loss = nca.loss(x=batch_x, labels=batch_y)
        print("Loss:", loss.item())

        # Backward pass and optimization
        # loss.backward()
        # optimizer.step()
        
        epoch_loss += loss.item() * len(batch_indices)
    

In [ ]:
print(transformed_x.min(), transformed_x.max())

In [ ]:
n = transformed_x.size(0)
sim_matrix = torch.zeros((n, n))

mu_transformed = generate_capacity(enumerate_permute(transformed_x[0].unsqueeze(0))[0])


In [ ]:
for i in range(n):
    for j in range(n):
        if i != j:  # Exclude self-similarity
            sim_matrix[i, j] = S1(transformed_x[i].unsqueeze(0), transformed_x[j].unsqueeze(0), mu_transformed).score()

In [ ]:
test = sim_matrix.numpy()
import seaborn as sns
sns.set(style="white")
plt.figure(figsize=(10, 8))
sns.heatmap(test, fmt=".2f", cmap='coolwarm')

In [ ]:
mask = torch.eye(n, dtype=torch.bool)
sim_matrix.masked_fill_(mask, float('-inf'))  # Set diagonal to -inf

In [ ]:
pij = torch.softmax(sim_matrix, dim=1)

In [ ]:
test = pij.numpy()
import seaborn as sns
sns.set(style="white")
plt.figure(figsize=(10, 8))
sns.heatmap(test, fmt=".2f", cmap='coolwarm')